In [1]:
%load_ext autoreload
%autoreload 2
import random
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from Openseas_Methods import *
from pycoingecko import CoinGeckoAPI
from datetime import datetime,timedelta
import numpy as np
from scipy.stats import pearsonr,spearmanr
import pickle
import decision_tree as tree
from itertools import islice
from decision_tree import get_eth_price
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.NFTDB
collection_data = db.salesCollection
stats_collection = db.NFTStats
slug_collection = db.completedSlugs
combos = db.NFT
skip_list=['ens','base-introduced','fundrop-pass','gemesis','apecoin','dai-stablecoin','uniswap','1inch-token']
address_type = 'seller_address'

In [3]:
records = collection_data.find({})

In [4]:
df = pd.DataFrame.from_records(records)

In [15]:
def dtformat(x):
    if x is None:
        return None
    return f"{x.day}-{x.month}-{x.year}"
def strpwrapper(x):
    if x is not None:
        return datetime.strptime(x,'%Y-%m-%dT%H:%M:%S')
    else:
        return None

In [16]:
df['parsed_date'] = df['timestamp'].apply(lambda x: strpwrapper(x))
df['date'] = df['parsed_date'].apply(lambda x:dtformat(x))

In [17]:
df['date']

0           2-9-2023
1           2-9-2023
2           2-9-2023
3           2-9-2023
4           1-9-2023
             ...    
3121362    26-2-2023
3121363    26-2-2023
3121364    26-2-2023
3121365    26-2-2023
3121366    26-2-2023
Name: date, Length: 3121367, dtype: object

In [20]:
# Group by 'group_column' and extract 'total_price' and 'timestamp' columns as lists
grouped_data = df.groupby('date').agg({
    'total_price': list,
    'timestamp': list
}).reset_index()
grouped_data_slug = df.groupby(['date','slug']).agg({
    'total_price': list,
    'timestamp': list
}).reset_index()
# Print the grouped data (optional)
grouped_data_slug

,date,slug,total_price,timestamp
0,1-1-2021,chromie-squiggle-by-snowfro,"[1.8, 0.7000000000000001]","[2021-01-01T21:09:39, 2021-01-01T01:45:09]"
1,1-1-2021,cryptocubes,"[0.35000000000000003, 0.5, 0.35000000000000003...","[2021-01-01T21:05:40, 2021-01-01T20:30:13, 202..."
2,1-1-2021,town-star,"[0.09100000000000001, 0.08, 0.0750000000000000...","[2021-01-01T23:27:18, 2021-01-01T22:44:33, 202..."
3,1-1-2022,0n1-force,"[1.0, 1.7500000000000002, 1.0, 1.1, 1.3, 0.980...","[2022-01-01T23:39:16, 2022-01-01T23:38:59, 202..."
4,1-1-2022,10ktf,"[0.13, 0.2, 0.17250000000000001, 0.32, 0.15000...","[2022-01-01T21:49:44, 2022-01-01T21:32:45, 202..."
...,...,...,...,...
116109,nan-nan-nan,town-star,"[0.93, 0.649]","[None, None]"
116110,nan-nan-nan,tubby-cats,"[0.267, 0.7000000000000001]","[None, None]"
116111,nan-nan-nan,unitedplanets,[0.8],[None]
116112,nan-nan-nan,veefriends-series-2,[1.0],[None]


In [31]:
slugs = df['slug'].unique()
dates = df['date'].unique()
for slug in slugs:
    x = []
    y = []
    for date in dates:
        slug_rows = grouped_data_slug[(grouped_data_slug['slug'] == slug) & (grouped_data_slug['date'] == date)]
        try:
            slug_prices = list(slug_rows['total_price'])[0]
        except:
            continue
        slug_avg = sum(slug_prices)/len(slug_prices)
        date_rows = grouped_data[(grouped_data['date'] == date)]
        date_prices = list(date_rows['total_price'])[0]
        date_avg = sum(date_prices)/len(date_prices)
        x.append(slug_avg)
        y.append(date_avg)
    print(slug,pearsonr(x,y))
    

atsnft PearsonRResult(statistic=-0.06763172674682238, pvalue=0.13176012562774383)
akidcalledbeast PearsonRResult(statistic=-0.16727442277639307, pvalue=0.010538297117684623)
opepen-edition PearsonRResult(statistic=0.06502527682265026, pvalue=0.3178250890312786)
archive-edition PearsonRResult(statistic=0.09623504646197036, pvalue=0.1719880641935977)
nakamigos PearsonRResult(statistic=-0.1362863512753136, pvalue=0.0790571612760644)
byopill PearsonRResult(statistic=-0.020310089792839253, pvalue=0.7435095317576775)
vv-checks PearsonRResult(statistic=-0.13339671885847995, pvalue=0.03770724274343902)
skyborne-genesisimmortals PearsonRResult(statistic=-0.09913465615894114, pvalue=0.5121602128613933)
max-pain-and-frens-by-xcopy PearsonRResult(statistic=0.1259269489440743, pvalue=0.0031205006046239546)
coolpetsnft PearsonRResult(statistic=0.7519317792342823, pvalue=9.431059460744045e-112)
mfers PearsonRResult(statistic=0.020332608630051072, pvalue=0.7338858014281089)
murakami-flowers-2022-offic

ValueError: x and y must have length at least 2.